# Lab Week 3: Line Search Methods + Conjugate Gradients

_Tutors: Andreas Grothey, Josh Fogg_

31 January 2025

This lab session experiments with the concepts of the lectures in weeks 2 and 3: line search methods and conjugate gradients. There is again an Assessed Task at the end (which follows on from Task 2(d)).

## 1. Speed of convergence

The first task is to investigate the speed of convergence of various iterations.

### (a)

The python file pi `approx.py` provides three different iterative schemes to approximate the value of $\pi$. These can be used by (for example)

In [ ]:
from pi_approx import pi_iter1_arctan as pi1
from pi_approx import pi_iter2_archim as pi2
from pi_approx import pi_iter3_borwein as pi3

These functions take one argument ($n$) and return the first $n$ elements of a sequence converging to $\pi$. s a python list. The following, for example, assigns the first 10 elements of the sequence generated by `pi_iter1_arctan` to `seq` (as a `np.array`).

In [ ]:
import numpy as np
seq = np.array(pi1(11))

We can now do something like

In [ ]:
(seq[1:11]-np.pi)/(seq[0:10]-np.pi)

to obtain the ratios $(x_{k+1} − x^∗)/(x_k − x^∗)$ for the first 10 elements of the sequence.

### (b)

Work out the speed of convergence for each of the three sequences.

### (c)

Have a look at the code for each function to see what calculations are involved. Would there be any circumstances where the first function (`pi_iter1_arctan`) might be preferable over the other two?

## 2. Line search methods

For most of this section we experiment with the files `GenericLineSearchMethod.py` and `LineSearchPlot.py`. Have a look at the file `GenericLineSearchMethod.py`.

In [ ]:
# This plots the iterates of the generic line search method for a given
# function on the contour plot of the function

import numpy as np
import matplotlib.pyplot as plt

# from himmelblau import himmelblau
from rosenbrock import rosenbrock
from ex21func import ex21
from ex46func import ex46

from GenericLineSearchMethod import GLSM

This defines a function `GLSM` that implements a generic line search method. It takes three arguments:

1. the starting point `x0` (as `np.array`),
2. the function to be optimised `func` and
3. a convergence tolerance `tol`.

The function code starts with a parameter section where you can choose different search directions:

- coordinate descent,
- steepest gradient,
- conjugate gradients, and
- four line search strategies:
  1. Exact,
  1. Armijo,
  2. Wolfe, and
  3. the $1/k$ rule.
 
Specific line search parameters such as $c_1$, $c_2$ can also be set there. The function `GLSM` returns a list of iterates encountered by the method to the calling function.

Now inspect the file `LineSearchPlot.py`: Most of this should look familiar. It allows you to choose from various functions to be optimised, after setting a few function parameters (like starting point and range of $x$ and $y$ values) and plot parameters it proceeds to create a contour plot of the selected function (much in the same way as we have done in the first Lab session).

### (a) Try Steepest Descent with exact line searches

By default (as you download them) the files are set up to optimise the function from Example 2.1 in the lectures ($f(x, y) = x^4 +2x^3 +2x^2 +y^2 −2xy$) by using the steepest descent method with exact line searches from $x_0 = (0.5, 1)^{\top}$. Run the python below and look at the plotted graph. You should observe that indeed subsequent search directions are perpendicular
to each other.

In [ ]:
# ------------------ parameters for the method --------------------------

function = "Ex21"  # can be "Rosenbrock", "Ex21", "Ex46", "Himmelblau"
tol = 1e-4  # tolerance for the line search method
n = 100  # number of points for the contour discretization
nl = 50  # number of levels for the contour plot

# ------------------ end parameters for the method --------------------------

if function == "Rosenbrock":
    func = rosenbrock
    x0 = np.array([0, 0])  # starting point (Rosenbrock)
    # after this come limits for the contour plot
    lmt_xlo, lmt_xup = -0.1, 1.1  # these are for Rosenbrock
    lmt_ylo, lmt_yup = -0.1, 1.1
elif function == "Ex21":
    func = ex21
    # x0 = np.array([1, 1])  # starting point (Ex2.1)
    x0 = np.array([0.5, 1])  # starting point (Ex2.1)
    # lmt_xlo, lmt_xup = -0.6, 1 # these are for Ex2.1 (up to nearest min)
    # lmt_ylo, lmt_yup = -0.4, 1.2
    lmt_xlo, lmt_xup = -1.2, 1  # these are for Ex2.1 (covering both min)
    lmt_ylo, lmt_yup = -1.2, 1.2
elif function == "Ex46":
    func = ex46
    x0 = np.array([2, 2])  # starting point (Ex4.6)
    lmt_xlo, lmt_xup = -1.2, 2.1  # these are for Ex4.6
    lmt_ylo, lmt_yup = -1.2, 2.1
elif function == "Himmelblau":
    raise ValueError("Himmelblau not supported yet")
    # func = himmelblau
    x0 = [1, 2]  # starting point
    lmt_xlo, lmt_xup = -1.2, 2.1  # these are for Ex4.6
    lmt_ylo, lmt_yup = -1.2, 2.1
else:
    raise ValueError("Function code not recognized")


# - - - - - - - - - This is the code for the contour plot - - - - - - -
plt.ion()
x_values = np.linspace(lmt_xlo, lmt_xup, n)
y_values = np.linspace(lmt_ylo, lmt_yup, n)
X, Y = np.meshgrid(x_values, y_values)

Z = func(0, X, Y)

# - - uncomment this to get contours with logarithmic progression
# lg_min = np.log((Z.min()))
# lg_max = np.log((Z.max()))
# levels = np.exp(np.linspace(lg_min, lg_max, 40))

fig, ax = plt.subplots(1, 1)
cp = ax.contour(X, Y, Z, nl)
# cp = ax.contour(X, Y, Z, levels)   # for logarithmic progression

# - - - - Calls the generic line search method and plots the path - - - -
path = GLSM(x0, func, tol)
ln = ax.plot(path[:, 0], path[:, 1])
# ln = ax.plot(path[:, 0], path[:, 1],'x-')

Experiment with different starting points and see what happens. 

Inspect the function `LineSearchExact.py` that implements the exact line search. While for simple functions it is sometimes easy to derive the step length for the exact line search it is substantially more difficult to do so for a general function. The method implemented here works by seeking an $\alpha^∗$ such that $\nabla f(x+\alpha^∗d)'d = 0$ (why?). It does so by first finding an interval $[\alpha_l, \alpha_u]$ such that $g_l = \nabla f(x+\alpha_{l}d)^{\top}d < 0$ and $g_u = \nabla f(x + \alpha_u d)^{\top}d > 0$ (while also requiring that $f(x + \alpha_{l}d) < f(x)$ and $f(x + \alpha_{u}d) < f(x)$) and then performs bisection to reduce the size of the interval. This is quite simplistic and results in a large number of necessary function evaluations. More sophisticated methods are possible, either by using second derivatives (like Newton’s method, see week 4), or various interpolations.

### (b) Improve Exact Line Search

One obvious place where the exact line search could be improved is in

```python
# by default just take the mid point between al and au
am = 0.5*(al+au)
```

from `LineSearchExact.py` where as part of the bisection the midpoint between $\alpha_l$ and $\alpha_u$ is used as the next trial point. The situation at this point is that we have $g_l < 0$ and $g_u > 0$ and are seeking a new $\alpha$ in $[\alpha_l, \alpha_u]$ with $g(\alpha) \approx 0$. Can you suggest a better formula (using $\alpha_l$, $\alpha_u$, $g_l$, $g_u$) to be used
instead of the midpoint? Try changing the line and see what happens.

### (c) Try coordinate descent

Now change to coordinate descent by selecting `direction = 'CD'` at the top of `GenericLineSearchMethod.py`. Try it from the same starting point $x_0 = (0.5, 1)$. You should find that the performance is much the same as for steepest descent.

### (d) Try Armijo and Wolfe line searches. Count function evaluations.

Go back to the steepest descent direction (`direction = 'SD'`) and change the line search to the back-tracking Armijo line search (`linesearch = 'Armijo'` – both changes in `GenericLineSearchMethod.py`). Use a parameter of $c_1 = 0.1$ (this should be the default). Again, run the code above an observe the pattern. Also inspect the number of iterations and function evaluations necessary to find the local minimum. You should find that the number of iterations does not change much while the number of function evaluations reduces drastically. Play around with different values for the $c_1$ parameter ($c_1 = 0.9$) gives an interesting plot. Please try to explain what happens here!. **Note that the assessed task follows on from this.**

If you want you can try the Bisection Line Search (for example with $c_1 = 0.1$, $x_2 = 0.5$), but the results will likely not be much different to the Armijo line search.

### (e) Pre-determined step sizes

As a final test on the test function `ex21` try the pre-determined step sizes (`linesearch='overk'`, "one-over-k"). Note that the maximal number of iterations is set to be 100 (you will hit that limit) and the final accuracy ($|g| = \ldots$) reached by the method.

### (f) Rosenbrock's function

For this task change the function that is optimised to Rosenbrock's function with starting point $x_0 = (0, 0)$. For this you need to change `function='Rosenbrock'` in the code cell above. I suggest you go back to the steepest descent direction with exact line searches. Rosenbrock's function is given by
$$
    f(x,y) = 100{(y-x^2)}^2 + {(1-x)}^2.
$$

It is a popular (and interesting) test function for unconstrained nonlinear optimization. The shape of a function describes a steep sided valley (with the valley floor along the parabola $y = x^2)$ with a slight gradient along the valley floor making $x^∗ = (1, 1)^{\top}$ the global minimizer (often described as a "banana function"). A line search method with a bad search direction will quickly hit the steep valley sides and make very slow progress. You should see that effect with the steepest descent method with exact line searches, that makes only little progress within the default maximum of 100 iterations (needing a very large number of function evaluations to do so). Using the Armijo-linesearch ($c_1 = 0.1$) is markedly better, but still does not reach the minimum within 100 iterations. You can play around with the Armijo and Wolfe line search and different parameters. Also try the pre-determined step sizes.

### (g) Conjugate Gradients

Finally try the Conjugate Gradient Descent method for Rosenbrock's functions (use `direction='CG'`) and see what happens.

## 3. Quadratics and Eigenvalues

I suspect that the tasks above will keep you busy for the lab session, but if time (or at home) you may want to try the following to get an intuitive feel of the shape of a quadratic function depending on the Hessian matrix (specifically its eigenvalues and eigenvectors).

### (a)

The python file `simplequadratic.py` defines a quadratic function $q(x) = \frac{1}{2}x^{\top}Qx + b^{\top}x$ in the sam way as other functions we have seen last week with the addition that it takes keyword arguments `Q=...` and `b=...` (as `np.arrays`) that specify the parameters of the quadratic. That means the following piece of code will plot the contours of
$$
    q(x) = x^{\top}\begin{bmatrix}
        3 & 2 \\
        2 & 3
    \end{bmatrix}x.
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from simplequadratic import simplequad as func

x_values = np.linspace(-2, 2, 100)
y_values = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x_values, y_values)
Q = np.array([[3, 2], [2, 3]])
Z = func(0, X, Y, Q=Q)
fig, ax=plt.subplots(1, 1)
ax.contour(X, Y, Z, 50)

### (b)

You can calculate the eigenvalue and eigenvectors of the matrix $Q$ in python by

In [ ]:
import numpy.linalg as LA
w, v = LA.eig(Q)
v0 = v[:,0]
v1 = v[:,1]

which gives the eigenvalues in `w` and the two eigenvectors as `v0`, `v1`. What does that give you for the
above matrix. Can you give a geometric interpretation of the eigenvectors?

### (c)

Also do this for the following matrices $Q$:

$$
    Q_2 = \begin{bmatrix}
        10 & 0 \\
        0 & 1
    \end{bmatrix},\quad Q_3 = \begin{bmatrix}
        3 & 1 \\
        1 & 4
    \end{bmatrix},\quad Q_4 = \begin{bmatrix}
        3 & 5 \\
        5 & 4
    \end{bmatrix},\quad Q_5 = \begin{bmatrix}
        -1 & 0 \\
        0 & -2
    \end{bmatrix}
$$
You may also want to try a 3d-surface plot.

### (d)

With the following lines of code you can plot the eigenvectors in the same diagram as the contour plot

```python
plt.arrow(0, 0, v0[0], v0[1], color=’r’, head_width=0.05)
plt.arrow(0, 0, v1[0], v1[1], color=’r’, head_width=0.05)
```

The function `plt.arrow` takes as arguments the origin point of the arrow $(0, 0)$ and the direction of the arrow ($v_0$). Again try this for the above matrices.

### (e)

What changes with the geometric interpretation if $b$ is no longer equal to the zero vector?

### (f)

Can you give a geometric description of the form of the function
$$
    q(x) = \frac{1}{2}x^{\top}\begin{bmatrix}
            2 & -1 & 0 \\
            -1 & 2 & 0 \\
            0 & 0 & -3
        \end{bmatrix}x
$$

## 4. Assessed Task

### (a)

> Determine the rate of convergence for the steepest descent method with Armijo line search ($c_1 = 0.1$) applied to the function from Example 2.1 (this was task 2(d) above). Does it match the prediction from the theorem given in the lecture regarding the convergence rate of steepest descent (Theorem 3.4 in Nocedal/Wright).
> 
> Note: You may find the method `np.linalg.norm(X, axis=1)` useful. It returns the 2-norm for every row in a $n\times2$ `np.array` `X`.

### (b)

> Explain what happens for 2(d) and $c_1 = 0.9$.

Please submit a document (or scan of a handwritten solution) that gives your answer to (a) and (b). You can include a piece of python code and its output to show what you did for part (a). You can upload this as a separate file or copy/paste it into a text/word/pdf file.

Please do not submit python code as Jupyter notebook (`*.ipynb`) or any zip files (`.zip`) since these don't display inline in Learn when we are marking the assignment. Plain python (`*.py`) is fine.

Submission is on the NLO Learn pages (under Assessments) by **Friday 7 February 10am**.